# Example

This page shows a few code examples using jupyter_rfb. Note that you need a server to generate the images (that's the whole point), so the output is not shown here.

Also see the [examples in the Github repo](https://github.com/vispy/jupyter_rfb/tree/main/examples).

In [ ]:
import numpy as np
import jupyter_rfb

In this first example, we implement `get_frame()` to produce an image.

In [ ]:
class HelloWorld1(jupyter_rfb.RemoteFrameBuffer):

    def get_frame(self):
        a = np.zeros((100, 100, 3), np.uint8)
        a[20:-20,20:-20,1] = 255
        return a

w = HelloWorld1()
w

Let's make it a bit more advanced, by keeping track of the widget size, so that we can provide an array with matching shape. We also take pixel_ratio into account, in case this is a hidpi display, or when the user has used the browser's zoom.

In [ ]:
class HelloWorld2(jupyter_rfb.RemoteFrameBuffer):
    
    def handle_event(self, event):
        if event["event_type"] == "resize":
            self._size = event
        
    def get_frame(self):
        w, h, r = self._size["width"], self._size["height"], self._size["pixel_ratio"]
        physical_size = int(h*r), int(w*r)
        a = np.zeros((physical_size[0], physical_size[1], 3), np.uint8)
        margin = int(20 * r)
        a[margin:-margin,margin:-margin,1] = 255
        return a

w = HelloWorld2(css_width='400px')
w

Let's wrap this up with a simple drawing app.

In [ ]:
class Drawingapp(jupyter_rfb.RemoteFrameBuffer):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.array = np.zeros((1, 1, 4), np.uint8)
        self.pen_colors = [(1, 0.2, 0, 1), (0, 1, 0.2, 1), (0.2, 0, 1, 1)]
        self.pen_index = 0        
    
    def handle_event(self, event):
        event_type = event.get("event_type", None)
        if event_type == "resize":
            # Take pixel ratio into account, use 8x less pixels (in each dim)
            self.pixel_ratio = event["pixel_ratio"] / 8
            w = int(event["width"] * self.pixel_ratio)
            h = int(event["height"] * self.pixel_ratio)
            self.array = np.zeros((h, w, 4), np.uint8)
        if event_type == "pointer_down":
            if event["button"] == 1:
                # Draw
                x = int(event["x"] * self.pixel_ratio)
                y = int(event["y"] * self.pixel_ratio)
                self.array[y, x] = 255 * np.array(self.pen_colors[self.pen_index])
                self.request_draw()
            elif event["button"] == 2:
                # Toggle color
                self.pen_index = (self.pen_index + 1) % len(self.pen_colors)
    
    def get_frame(self):
        return self.array

w = Drawingapp(css_width="600px", css_height="400px")
w